In [29]:
print("도커파일 메이커입니다.")
user_name = input("사용자 명을 입렵해주세요.")
lib_list = []
print("사용하고자 하는 library 및 버전을 올려주세요. 나중에도 추가 가능합니다.")
with open("./library.csv", "r") as f:
    lines = f.readlines()
    for line in lines:
        lib_list.append(line.rstrip(',').split(','))

for lib in lib_list:
    print(lib)

도커파일 메이커입니다.
사용자 명을 입렵해주세요.
사용하고자 하는 library 및 버전을 올려주세요. 나중에도 추가 가능합니다.
['numpy', '\n']
['asdf', '1.21\n']


In [28]:
file_pre =  """FROM ubuntu:latest

RUN apt-get update && yes|apt-get upgrade
RUN apt-get install -y emacs

RUN apt-get install -y wget bzip2

RUN apt-get -y install sudo
RUN apt-get -y install python3
RUN apt-get -y install python3-pip

RUN adduser --disabled-password --gecos '' ubuntu
RUN adduser ubuntu sudo
RUN echo '%sudo ALL=(ALL) NOPASSWD:ALL' >> /etc/sudoers
USER ubuntu
WORKDIR /home/ubuntu/
RUN chmod a+rwx /home/ubuntu/

RUN wget https://repo.continuum.io/archive/Anaconda3-5.0.1-Linux-x86_64.sh
RUN bash Anaconda3-5.0.1-Linux-x86_64.sh -b
RUN rm Anaconda3-5.0.1-Linux-x86_64.sh
RUN sudo apt install python3-dev python3-pip
RUN sudo pip3 install -U virtualenv
RUN export PATH=~/anaconda3/bin:$PATH
RUN conda create-y -n venv pip python=3.7
RUN source activate venv"""
#################################################
file_suf = """
ENV PATH /home/ubuntu/anaconda3/bin:$PATH

RUN conda update conda
RUN conda update anaconda
RUN conda update --all

RUN mkdir /home/ubuntu/notebooks
RUN jupyter notebook --generate-config --allow-root
RUN echo "c.NotebookApp.password = u'sha1:6a3f528eec40:6e896b6e4828f525a6e20e5411cd1c8075d68619'" >> /home/ubuntu/.jupyter/jupyter_notebook_config.py

EXPOSE 8888

CMD ["jupyter", "notebook", "--allow-root", "--notebook-dir=/home/ubuntu/notebooks", "--ip='*'", "--port=8888", "--no-browser"]
"""
file_lib = "\n"
for i in range(len(lib_list)):
    code = "RUN pip3 install " + ("==").join(lib_list[i]) + "\n"
    file_lib += code
    
file = file_pre + file_lib + file_suf

print (file)
with open("./Dockerfile", "w") as f:
    f.write(file)

FROM ubuntu:latest

RUN apt-get update && yes|apt-get upgrade
RUN apt-get install -y emacs

RUN apt-get install -y wget bzip2

RUN apt-get -y install sudo

RUN adduser --disabled-password --gecos '' ubuntu
RUN adduser ubuntu sudo
RUN echo '%sudo ALL=(ALL) NOPASSWD:ALL' >> /etc/sudoers
USER ubuntu
WORKDIR /home/ubuntu/
RUN chmod a+rwx /home/ubuntu/

RUN wget https://repo.continuum.io/archive/Anaconda3-5.0.1-Linux-x86_64.sh
RUN bash Anaconda3-5.0.1-Linux-x86_64.sh -b
RUN rm Anaconda3-5.0.1-Linux-x86_64.sh
RUN pip install numpy==3.35

RUN pip install keras

ENV PATH /home/ubuntu/anaconda3/bin:$PATH

RUN conda update conda
RUN conda update anaconda
RUN conda update --all

RUN mkdir /home/ubuntu/notebooks
RUN jupyter notebook --generate-config --allow-root
RUN echo "c.NotebookApp.password = u'sha1:6a3f528eec40:6e896b6e4828f525a6e20e5411cd1c8075d68619'" >> /home/ubuntu/.jupyter/jupyter_notebook_config.py

EXPOSE 8888

CMD ["jupyter", "notebook", "--allow-root", "--notebook-dir=/home/ubuntu/n